## PS 6
## Estimation of Variance For Multiple Segments

Previously we've found that $\sigma_I$ for single segment can be estimated using the following relation.

$$\sigma_I=\frac{1}{12\sqrt{5}}\frac{(b-a)^3}{\sqrt{N}}g''\left(\frac{a+b}{2}\right)$$

where $g=\frac{f}{w}$. For multiple segments, we sum up the variance for each segment and find the standard deviation.

Then the relation becomes 

$$\sigma_I=\frac{1}{12\sqrt{5}}\frac{(b-a)^3}{M^2\sqrt{N}}\sqrt{\sum_i \frac{1}{M}g''\left( x_{mid}^{(i)}\right)}$$,

or,

$$\sigma_I=\frac{1}{12\sqrt{5}}\frac{(b-a)^3}{M^2\sqrt{N}}g''_{rms}$$,

where $M$ is the number of segments. However this formula is not practical since $g$ is calculated using different weight functions. Therefore we would like to write this down in terms of $f$. Going back to the relation for one segment, instead of $g''$ we can write,

$$g''=\frac{f''-2f'w'w+2fw'^2}{kw^3}$$. 

Making the appropriate assumptions, we can also write,

$$g''\simeq \frac{f''}{kw^3} $$.

where $w=f(a)+\frac{f(b)-f(a)}{b-a}(x-a)$ and $\frac{1}{k}=(b-a)\frac{f(a)+f(b)}{2}$.

This leads to $g''=\frac{f''}{w^2}$

Note that all functions are evaluated at the middle point unless specified otherwise. Realizing that $w(\frac{f(a)+f(b)}{2})$.

We are left with $\frac{4f''}{(f(a)+f(b))^2}$.

Then the relation for single segment becomes,

$$\sigma_I=\frac{1}{3\sqrt{5}}\frac{(b-a)^4}{\sqrt{N}(f(a)+f(b))^2}f''\left(\frac{a+b}{2}\right)$$


Let's test this result.



In [1]:
import numpy as np
from numpy.polynomial import Polynomial as P
#import plotly
#import plotly.plotly as py
#import plotly.figure_factory as ff
import matplotlib.pyplot as plt
#Integrand function
def f(x,H):
    return (x-5)*np.exp(-(x/2-3))+H

#Calculates the coefficients of linear weight function.    
def findw(f,H,lower,upper,normalize):
    #Find the linear function.
    slope=(f(upper,H)-f(lower,H))/(upper-lower)
    a=slope
    b=-slope*upper+f(upper,H)
    #Normalization.
    A=(a/2)*(upper**2)+b*upper-(a/2)*(lower**2)-b*lower
    if normalize:
        a/=A
        b/=A
    return [a,b]
#Performs integration.
def integrate(f,lower,upper,N,C):
    H=C
    w=findw(f,H,lower,upper,True)    
    #Generate uniform random inputs.
    inputs=np.random.rand(N)    
    a=w[0]/2  
    b=w[1]
    c=-(a*lower**2+b*lower)
    
    SUM=0
    SUM2=0
    
    inverse_inputs=[]
    for i in inputs:
        p=[(-b-np.sqrt(b**2-4*a*(c-i)))/(2*a),(-b+np.sqrt(b**2-4*a*(c-i)))/(2*a)]
        if p[0]>=lower and p[0]<=upper:
            inverse_inputs.append(p[0])
        else :
            inverse_inputs.append(p[1])

    inverse_inputs=np.array(inverse_inputs)
    #Calculate f(inverse(x))/w(inverse(x)).
    outputsF=f(inverse_inputs,H)
    outputsW=w[0]*(inverse_inputs)+w[1]
    outputs=outputsF/outputsW
    SUM=outputs.sum()
    SUM2=(outputs*outputs).sum()
    var=SUM2/N-(SUM/N)**2
    var=var/N
    #Store generated points for variance calculation.
    Vsum=outputs.sum()
    return Vsum/N-H*(upper-lower),(upper-lower)**2*var
    



def theoretical_sigma(f,lower,upper,N,C):
    
    w=findw(f,C,lower,upper,True) 
    
    flower=f(lower,C)
    fmiddle=f((lower+upper)/2,C)
    fupper=f(upper,C)
    
    wlower=w[0]*lower+w[1]
    wmiddle=w[0]*(lower+upper)/2+w[1]
    wupper=w[0]*upper+w[1]
    
    gupper=fupper/wupper
    gmiddle=fmiddle/wmiddle
    glower=flower/wlower    

    
   # sigma=np.abs(0.03727*(upper-lower)**3/np.sqrt(N)*(4*(gupper-2*gmiddle+glower)/(upper-lower)**2))
    sigma=np.abs(1/np.sqrt(45)*(upper-lower)**4/np.sqrt(N)*1/(flower+fupper)**2*(4*(fupper-2*fmiddle+flower)/(upper-lower)**2))
    
    #return (upper-lower)*np.sqrt(var)
    return sigma

low=4.1
up=4.2
#Divide the region into 10 pieces.
l=[low,up]   
#Real value of the integral
I_real=-.12002
#N values
N=[100,500,1000,5000,10000,50000,100000]
#Integration results.
results=[]
#Standart deviation values
sigmas=[]
theory_sigmas=[]
print('N','   sigma_exp','       sigma_theory','    sigma_theory/sigma_exp')

for k in N:
    I=0
    sigma=0
    S=1000
    for i in range (0,len(l)-1):
        temp_sigma=[]
        for j in range(0,10):
            I=0
            sigma=0
            temp,temp2=integrate(f,l[i],l[i+1],k,S)
            I+=temp
            sigma+=temp2
            temp_sigma.append(np.sqrt(sigma))
    results.append(I)
    sigmas.append(np.mean(temp_sigma))
    theory_sigmas.append(theoretical_sigma(f,low,up,k,S))
    sigma=np.mean(temp_sigma)
    print(k,sigma,theoretical_sigma(f,low,up,k,S),theoretical_sigma(f,low,up,k,S)/sigma)    

N    sigma_exp        sigma_theory     sigma_theory/sigma_exp
100 1.11460850492e-06 1.14458062899e-12 1.02689027039e-06
500 5.09567402969e-07 5.11872018431e-13 1.00452269013e-06
1000 3.57654069792e-07 3.61948175332e-13 1.01200630974e-06
5000 1.61893449991e-07 1.61868144875e-13 9.99843692774e-07
10000 1.14201020892e-07 1.14458062899e-13 1.00225078554e-06
50000 5.1039572801e-08 5.11872018431e-14 1.00289244275e-06
100000 3.60304466848e-08 3.61948175332e-14 1.00456199863e-06


It seems the result is correct. Let's apply the same relation for each segment and add it up.

This gives,

$$\sigma_I=\frac{1}{3\sqrt{5}}\frac{(b-a)^4}{M^3\sqrt{N}}\sum_ i \frac{1}{M}\frac{f''\left(\frac{a_i+b_i}{2}\right)}{(f(a_i)+f(b_i))^2}$$
